In [1]:
import pandas as pd
import numpy as np
import sys

In [2]:
import transaction_demo

In [3]:
seed = 42

### Load transactions

In [4]:
df = pd.read_csv('../data/transactions.csv')
df.head(10)

,Unnamed: 0,trans_id,account_id,date,type,operation,amount,balance,k_symbol,bank,account,mean_income
0,0,695247,2378,1993-01-01,1,2.0,700.0,700.0,NaN,NaN,NaN,[ 11271.53754941]
1,1,695250,2378,1993-01-22,1,2.0,7268.0,7968.0,NaN,NaN,NaN,[ 11271.53754941]
2,2,695266,2378,1993-01-23,1,2.0,14440.0,22408.0,NaN,NaN,NaN,[ 11271.53754941]
3,3,695262,2378,1993-01-24,1,2.0,49752.0,72160.0,NaN,NaN,NaN,[ 11271.53754941]
4,4,695265,2378,1993-01-26,1,2.0,21899.0,94059.0,NaN,NaN,NaN,[ 11271.53754941]
5,5,695560,2378,1993-01-31,0,3.0,34700.0,59453.7,NaN,NaN,NaN,[ 11271.53754941]
6,6,3608592,2378,1993-01-31,1,NaN,94.7,94153.7,2,NaN,NaN,[ 11271.53754941]
7,7,695272,2378,1993-02-05,1,2.0,30192.0,89645.7,NaN,NaN,NaN,[ 11271.53754941]
8,8,695340,2378,1993-02-07,0,0.0,9612.0,80033.7,4,EF,1222903.0,[ 11271.53754941]
9,9,695270,2378,1993-02-11,1,2.0,31038.0,111071.7,NaN,NaN,NaN,[ 11271.53754941]


### Clean the dataset

* Leave target columns only (date, operation and amount)
* filter out NaNs
* convert operations to int

In [5]:
df = transaction_demo.clean_dataset(df)
df.head(10)

,account_id,date,operation,amount
0,2378,1993-01-01,2,700.0
1,2378,1993-01-22,2,7268.0
2,2378,1993-01-23,2,14440.0
3,2378,1993-01-24,2,49752.0
4,2378,1993-01-26,2,21899.0
5,2378,1993-01-31,3,34700.0
7,2378,1993-02-05,2,30192.0
8,2378,1993-02-07,0,9612.0
9,2378,1993-02-11,2,31038.0
10,2378,1993-02-11,3,25200.0


Leave transaction only for the 1993 year:

In [6]:
df = df[pd.to_datetime(df['date']) < pd.to_datetime('1994-01-01')]

### Segment all transaction by month

We consider each month of an account as a separate segment. All segments are considered the same. It means our generator will generate a monthly snapshot of transactions for some user.

In [7]:
df = transaction_demo.segment_by_month(df)
df.head(10)

,operation,amount,day,segment_id
0,2,700.0,1,2378:1993-01
1,2,7268.0,22,2378:1993-01
2,2,14440.0,23,2378:1993-01
3,2,49752.0,24,2378:1993-01
4,2,21899.0,26,2378:1993-01
5,3,34700.0,31,2378:1993-01
7,2,30192.0,5,2378:1993-02
8,0,9612.0,7,2378:1993-02
9,2,31038.0,11,2378:1993-02
10,3,25200.0,11,2378:1993-02


### Prepare training data

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
transform_pipeline = Pipeline([
    ("vectorizer", transaction_demo.TransactionVectorizer(
        group_column='segment_id',
        value_column='amount',
        dim1_column='operation',
        dim2_column='day',
        dim1_size=5,
        dim2_size=32)),
    ("scale", StandardScaler())
])

In [9]:
df_transformed = transform_pipeline.fit_transform(df)

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(df_transformed, test_size=0.3, random_state=seed)

### Train the model

In [11]:
from synthesized.modules.synth import AlphaSynth
from sklearn.metrics import mean_squared_error

# define the estimator. We will use L2 penalty to augment the generalizability of the encoder
alpha = AlphaSynth(n_epochs=100, n_hidden=250, learning_rate=0.01, batch_size=106, 
                 display_step=10, activation_function='relu', verbose=2, min_change = 1e-6, 
                 random_state=seed, clip=True, l2_penalty=1e-5, 
                 early_stopping=True)

# fit
alpha.fit(X_train)

# transform and reconstruct the test images
reconstructed = alpha.feed_forward(X_test)

# get the error:
mse = ((X_test - reconstructed) ** 2).sum(axis=1).sum() / X_test.shape[0]
print("\nTest MSE: %.4f" % mse)

Epoch: 1, cost=0.279351, time=0.1933 (sec)
Epoch: 11, cost=0.035733, time=0.2362 (sec)
Epoch: 21, cost=0.035093, time=0.1611 (sec)
Epoch: 31, cost=0.036756, time=0.1527 (sec)
Epoch: 41, cost=0.027045, time=0.1708 (sec)
Epoch: 51, cost=0.035427, time=0.2167 (sec)
Epoch: 61, cost=0.050448, time=0.1568 (sec)
Epoch: 71, cost=0.031682, time=0.2732 (sec)
Epoch: 81, cost=0.030665, time=0.2832 (sec)
Epoch: 91, cost=0.034699, time=0.2835 (sec)
Optimization complete after 100 epoch(s). Average epoch time: 0.2273 seconds

Test MSE: 8.0422


### Convert generated data to transactions

In [15]:
df_orig = transform_pipeline.inverse_transform(X_test).drop(columns=['segment_id'])

In [16]:
df_synth = transform_pipeline.inverse_transform(reconstructed).drop(columns=['segment_id'])

In [17]:
df_orig.head(10)

,day,operation,amount
0,1,2,26258.0
1,3,3,3200.0
2,1,1,6550.0
3,3,4,16646.0
4,3,6,11900.0
5,1,9,14.6
6,0,2,3016.0
7,2,2,8720.0
8,4,4,18467.0
9,2,7,9100.0


In [18]:
df_synth.head(10)

,day,operation,amount
0,0,1,139.897339
1,1,1,119.069122
2,2,1,116.029465
3,3,1,117.953697
4,4,1,139.312607
5,0,2,583.938599
6,1,2,29051.906250
7,2,2,783.110291
8,3,2,765.452454
9,4,2,747.994690


### Linkage attack test

In [21]:
from synthesized.testing.linkage_attack import linkage_attack, t_closeness_check

Find vulnerabilities in the original dataset:

In [32]:
vulnerabilities_orig = t_closeness_check(df_orig)

The total number of vulnerabilities:

In [33]:
len(vulnerabilities_orig)

8569

Find the remaining vulnerabilities in the synthetical data:

In [34]:
vulnerabilities_remaining = linkage_attack(df_orig, df_synth, categ_columns={'operation', 'day'}, t_closeness=0.2, k_distance=0.8)

The number of remaining vulnerabilities in synthetical data:

In [35]:
len(valnurabilities_remaining)

789

### Percent of remaining vulnerabilities:

In [44]:
print('{:.2f}%'.format(len(valnurabilities_remaining) / len(vulnerabilities_orig) * 100))

9.21%
